# Creating datatypes

In [1]:
from recordclass import dataobject
import sys

## LinkedList

In [2]:
class LinkedItem(dataobject, fast_new=True):
    __fields__ = 'val', 'next'
    
class LinkedList(dataobject):
    __fields__ = 'start', 'end'
    start = None
    end = None
        
    def append(self, val):
        link = LinkedItem(val, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link
        
    def __del__(self):
        curr = self.start
        
        while curr is not None:
            next = curr.next
            del curr
            curr = next

    def __iter__(self):
        return IterLinkedList(self.start)

class IterLinkedList(dataobject, fast_new=True):
    __fields__ = 'node',
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val


In [3]:
def make_llist(N):
    ll = LinkedList()
    for i in range(N):
        ll.append(i)
    return ll

In [4]:
class LinkedItem2:
    __slots__ = 'val', 'next'
    
    def __init__(self, val, next=None):
        self.val = val
        self.next = next

class LinkedList2:
    __slots__ = 'start', 'end'
        
    def __init__(self, start=None, end=None):
        self.start = start
        self.end = end

    def append(self, val):
        link = LinkedItem2(val, None)
        if self.start is None:
            self.start = link
        else:
            self.end.next = link
        self.end = link

    def __iter__(self):
        return IterLinkedList2(self.start)

class IterLinkedList2:
    __slots__ = 'node',
    
    def __init__(self, node):
        self.node = node
    
    def __next__(self):
        node =  self.node
        if node is None:
            raise StopIteration
        
        val = node.val
        self.node = node.next
        return val

In [5]:
def make_llist2(N):
    ll = LinkedList2()
    for i in range(N):
        ll.append(i)
    return ll

In [6]:
N = 100000
Mb = 1000000

In [14]:
ll1 = make_llist(N)

In [15]:
sum(ll1)

4999950000

In [16]:
M1 = sys.getsizeof(ll1) + N * sys.getsizeof(ll1.start)
print('Memory footprint:', M1/1000000, 'Мб')

Memory footprint: 3.200032 Мб


In [17]:
ll2 = make_llist2(N)

In [18]:
sum(ll2)

4999950000

In [19]:
M2 = sys.getsizeof(ll2) + N * sys.getsizeof(ll2.start)
print('Memory footprint:', M2/1000000, 'Мб')

Memory footprint: 4.800048 Мб


In [20]:
print(100*M1/M2)

66.66666666666667
